In [5]:
#Declarations******************************************************************************
from __future__ import print_function
import numpy as np
import os
import sys
import tarfile
import math
import random
from IPython.display import display, Image
from scipy import ndimage
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
from funcCNN import *
from crossValB import *
import matplotlib.pyplot as plt

dataFolder="../data/"
#Output:
f=open(dataFolder+'results/outputVector.txt', 'w')
f.write('1\n')
f.write('1\n')
temp=1.0
trueAcc=str(temp)
print(trueAcc)
f.write(trueAcc+'\n')
f.close()
#Parameters*******************************************************************************
#Maximum number of iterations
#iterMax=int(sys.argv[1])
iterMax=2000
#maximum number of iterations
limit=1.01
#regularization on the weights
beta=0.001
#version of the code
version='gen1'
#size of batch
batchSize=70
#Parameters*******************************************************************************
#w1=int(sys.argv[2]) #12
#w4=int(sys.argv[3]) #196
#h1=int(sys.argv[4]) #148
#wd1=int(sys.argv[5]) #21
#index=int(sys.argv[6]) #0
#kfoldIndex=int(sys.argv[7]) #0
#generation=int(sys.argv[8]) #0

w1=12
w4=196
h1=148
wd1=21
index=0
kfoldIndex=2
generation=0

#Input Data***********************************************************************************

#kfoldIndex=0, dataFolder='../data/'



(test_dataset_Flat,oneHot_test_labels,valid_dataset_Flat,oneHot_valid_labels,
	train_dataset_Flat,oneHot_train_labels,labelSize,vectorSize)=get_Info(
	kfoldIndex,dataFolder)
runs=int(len(oneHot_train_labels)/batchSize)
print(runs)


#Model declaration************************************************************************
#import tensorflow as tf
#declare interactive session
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
sess=tf.InteractiveSession()


#INPUT->CONV LAYER->CONV LAYER->CONV LAYER->RECT FLAT->RECT DROPOUT

#function to declare easily the weights only by shape
def weight_variable(shape):
	initial = tf.truncated_normal(shape, stddev=0.1)
	return tf.Variable(initial)
#function to declare easily the bias only by shape
def bias_variable(shape):
	initial = tf.constant(0.1, shape=shape)
	return tf.Variable(initial)

#input variable
x = tf.placeholder(tf.float32, [None, vectorSize])
#keep probability to change from dropout 0.50 to 1.0 in validation and test
keep_prob = tf.placeholder(tf.float32)
#expected outputs variable
y_ = tf.placeholder(tf.float32, [None, labelSize])

#arrange the tensor as an image (1*31029) 1 channel
x_image0 = tf.reshape(x, [-1,1,vectorSize,1])
x_image = tf.transpose(x_image0, perm=[0,3,2,1])
#arrange the tensor into 1 channels (1*31029)

#1 LAYER*************************************************************************************
#1 Convolutional Layer Explicit for regularization of the weights
#weigth first layer 1 input channels, 12 output channels, 1x21 filter window size
W_conv1 = weight_variable([1, wd1, 1, w1])
#bias declaration the size has to be the same as the output channels 12
b_conv1 = bias_variable([w1])
#convolution (input weights) moving 1 step each time with a relu
h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, 
	strides=[1, 1, 1, 1], padding='SAME') + b_conv1)
#max pooling with a 148 width window size, moving 148 in width by step
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 1, h1, 1],
	strides=[1, 1, h1, 1], padding='SAME')
#output=545/4
#1 LAYER*************************************************************************************

#Rectifier LAYER*****************************************************************************
#calculated coefficient for the flattening from the size of the 3 convolutional layer
coef=int (h_pool1.get_shape()[1]*h_pool1.get_shape()[2]*h_pool1.get_shape()[3])
h_pool2_flat = tf.reshape(h_pool1, [-1, coef])
#declare the weights considering the constants and 256 output 
W_fc1 = weight_variable([coef, w4])
b_fc1 = bias_variable([w4])

#rectifier (matmul)
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
#Rectifier LAYER*****************************************************************************

#Rectifier-Dropout LAYER**********************************************************************
#dropout
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
#declare weights with the ouput layer in this case 2 (labelSize)
W_fc2 = weight_variable([w4, labelSize])
b_fc2 = bias_variable([labelSize])
#output
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
#Rectifier-Dropout LAYER**********************************************************************

#Loss Function********************************************************************************
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[0]))
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_conv, labels=y_)+
	beta*tf.nn.l2_loss(W_conv1))
#Optimizer Adam at 1e-5 (literature)**********************************************************
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#softmax prediction remember we are using one hot labels
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))

trueResult=tf.argmax(y_conv,1)
trueTest=tf.argmax(y_,1)
#accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#Loss Function********************************************************************************
valid_accuracy_global=0.0
test_accuracy_global=0.0


#start
sess.run(tf.initialize_all_variables())
# Add ops to save and restore all the variables.
saver = tf.train.Saver()
#Extra to verify sizes************************************************************************
print(x_image.get_shape())
#print(h_conv1.get_shape())
print(h_pool1.get_shape())

print(y_conv.get_shape())
print(y_.get_shape())
#Extra to verify sizes************************************************************************

print('test ',str(oneHot_valid_labels.shape[0]))

xaV,yaV=getBatch(valid_dataset_Flat,oneHot_valid_labels,oneHot_valid_labels.shape[0],vectorSize)
print(xaV[1])
print(yaV[1])

yResult=[]
yTest=[]
#Ouput Data Variables*************************************************************************
name=str(str(kfoldIndex)+'_'+str(iterMax)+'_'+str(w1)+'_'+str(w4)+'_'+str(h1)+'_'+str(wd1))
f = open(dataFolder+'results/'+name+'.txt', 'a')
#Ouput Data Variables*************************************************************************

#Main Loop************************************************************************************
#initialize variables
iter=0
train_accuracy=0.0
valid_accuracy=0.0
test_accuracy=0.0
#best validation accuracy
best=0
validWindow=[0,0,0]
repeatWindow=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
validBest=1e6
#limit (normally a dummy value 0.80) and iterations...
while ((best<limit) & (iter<iterMax)):
	indexBatch=[]
	for iB in range (0,len(oneHot_train_labels)):
		indexBatch.append(iB)
	random.shuffle(indexBatch)
	for run in range (0,runs):
		#Get data from train set and store in xa (inputs),ya(labels),ca(constants) 100 batch size
		xa,ya=getBatch_run(train_dataset_Flat,oneHot_train_labels,batchSize,run,indexBatch,vectorSize)
		#pass the values and 0.50 dropout (literature)
		train_step.run(feed_dict={x: xa, y_: ya, keep_prob: 0.5})
		#print each 10 iterations
	#calculate train accuracy
	xa,ya=getBatch(train_dataset_Flat,oneHot_train_labels,batchSize,vectorSize)
	train_accuracy = accuracy.eval(feed_dict={
		x:xa, y_: ya, keep_prob: 1.0})
	#calculate validation accuracy
	xaV,yaV=getBatch(valid_dataset_Flat,oneHot_valid_labels,oneHot_valid_labels.shape[0],vectorSize)
	
	valid_accuracy=accuracy.eval(feed_dict={
		x: xaV, y_: yaV, keep_prob: 1.0})
	#calculate validation loss
	cross_entropyVal=cross_entropy.eval(feed_dict={
		x: xaV, y_: yaV, keep_prob: 1.0})
	#calculate train loss
	cross_entropyTrain=cross_entropy.eval(feed_dict={
		x: xa, y_: ya, keep_prob: 1.0})
	#append values for graphs
	#if valid accuracy is better than the best accuracy then calculate test accuracy
	#if valid_accuracy>best:
	validWindowValue=0
	tempValid=validWindow
	for i in range(0,len(validWindow)-1):
		tempValid[i]=validWindow[i+1]
	for i in range(0,len(validWindow)):
		validWindow[i]=tempValid[i]
	validWindow[len(validWindow)-1]=valid_accuracy
	for i in range(0,len(validWindow)):
		validWindowValue=validWindowValue+validWindow[i]
	validWindowValue=validWindowValue/len(validWindow)
	tempValid=repeatWindow
	for i in range(0,len(repeatWindow)-1):
		tempValid[i]=repeatWindow[i+1]
	for i in range(0,len(repeatWindow)):
		repeatWindow[i]=tempValid[i]
	repeatWindow[len(repeatWindow)-1]=valid_accuracy
	if np.var(repeatWindow)==0 and iter>10:
		iter=iter
	if (validWindowValue)>best or cross_entropyVal<validBest:
		validBest=cross_entropyVal
		best=(validWindowValue)
		
		xaT,yaT=getBatch(test_dataset_Flat,oneHot_test_labels,oneHot_test_labels.shape[0],vectorSize)
		#calculate test accuracy
		test_accuracy= accuracy.eval(feed_dict={x:xaT, 
			y_: yaT, keep_prob: 1.0})
		# Save the variables to disk.
		if (kfoldIndex==0):
			save_path = saver.save(sess, dataFolder+"model/model.ckpt")
		#plotNNFilter(units)
		#calculate the results of the whole model, probabilities in one hot format
		results=correct_prediction.eval(feed_dict={x:xaT, y_: yaT, keep_prob: 1.0})
		yResult=trueResult.eval(feed_dict={x:xaT, y_: yaT, keep_prob: 1.0})
		yTest=trueTest.eval(feed_dict={x:xaT, y_: yaT, keep_prob: 1.0})
		fOut=open(dataFolder+'results/outputVector.txt', 'w')
		fOut.write('1\n')
		fOut.write('1\n')
		temp=1.0-best
		trueAcc=str(temp)
		print(trueAcc)
		fOut.write(trueAcc+'\n')
		fOut.close()				
	#append everything to a log for retrieving results
	log="%d	%d	%g	%g	%g	%g	%g	%g"%(iter,kfoldIndex,train_accuracy,valid_accuracy,best,
		test_accuracy,cross_entropyVal,cross_entropyTrain)
	print(log)
	f.write(log+'\n')
	iter=iter+1
#Main Loop************************************************************************************

f.close()
saveVectorInt(dataFolder+'results/results'+name+'.txt',yResult)
saveVectorInt(dataFolder+'results/test'+name+'.txt',yTest)
f = open(dataFolder+'log3.txt', 'a')
name=str(str(index)+'_'+str(kfoldIndex)+'_'+str(iterMax)+'_'+str(test_accuracy)+'_'+str(valid_accuracy)+'_'+str(best)+'_'+str(w1)+'_'+
		 str(w4)+'_'+str(h1)+'_'+str(wd1)+'_'+str(generation))
f.write(name+'\n')
f.close()

#f=open(str(index)+'.index','a')
#name=str(str(index)+'_'+str(kfoldIndex)+'_'+str(iterMax)+'_'+str(test_accuracy)+'_'+str(valid_accuracy)+'_'+str(best)+'_'+str(w1)+'_'+#
#		 str(w4)+'_'+str(h1)+'_'+str(wd1)+'_'+str(generation))
#f.write(name+'\n')
#f.close()
#Input Data***********************************************************************************
#close session
f=open(dataFolder+'results/outputVector.txt', 'w')
f.write('1\n')
f.write('1\n')
temp=1.0-best
trueAcc=str(temp)
print(trueAcc)
f.write(trueAcc+'\n')
f.close()

xaT,yaT=getBatch(test_dataset_Flat,oneHot_test_labels,oneHot_test_labels.shape[0],vectorSize)

units = sess.run(h_conv1,feed_dict={x:xaT, 
			y_: yaT, keep_prob: 1.0})
print(units.shape)
units = sess.run(h_pool1,feed_dict={x:xaT, 
			y_: yaT, keep_prob: 1.0})
print(units.shape)

sess.close()


1.0
data set (840,)
vectorSize 1260
532
1260
(532,)
28
(28,)
280
(280,)
6
532
6
(532, 6)
28
6
(28, 6)
280
6
(280, 6)
7


2023-02-20 23:20:59.366123: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8332 MB memory:  -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:da:00.0, compute capability: 7.5


(?, 1, 1260, 1)
(?, 1, 9, 12)
(?, 6)
(?, 6)
test  28
[1.   0.5  0.75 ... 0.5  1.   1.  ]
[0. 0. 0. 0. 1. 0.]
0.952380950252215
0	2	0.228571	0.142857	0.047619	0.167857	1.97649	1.89558
0.9047619005044302
1	2	0.185714	0.142857	0.0952381	0.167857	1.91534	1.82864
0.8571428507566452
2	2	0.185714	0.142857	0.142857	0.167857	1.87287	1.7958
0.8571428507566452
3	2	0.157143	0.142857	0.142857	0.167857	1.84731	1.84034
0.8571428507566452
4	2	0.171429	0.142857	0.142857	0.167857	1.83112	1.78724
0.8571428507566452
5	2	0.0857143	0.142857	0.142857	0.167857	1.82172	1.82027
0.8571428507566452
6	2	0.142857	0.142857	0.142857	0.167857	1.81426	1.78146
0.8571428507566452
7	2	0.171429	0.142857	0.142857	0.167857	1.80783	1.79504
0.8571428507566452
8	2	0.157143	0.142857	0.142857	0.171429	1.80391	1.79025
0.8571428507566452
9	2	0.228571	0.142857	0.142857	0.167857	1.80247	1.78734
0.8571428507566452
10	2	0.185714	0.142857	0.142857	0.167857	1.79949	1.79271
0.8571428507566452
11	2	0.2	0.142857	0.142857	0.167857	1.79677	1.

128	2	0.257143	0.178571	0.22619	0.239286	1.80006	1.78279
129	2	0.342857	0.178571	0.22619	0.239286	1.80111	1.78031
130	2	0.214286	0.178571	0.22619	0.239286	1.80137	1.7813
131	2	0.328571	0.178571	0.22619	0.239286	1.80142	1.7803
132	2	0.271429	0.178571	0.22619	0.239286	1.79926	1.77943
133	2	0.271429	0.25	0.22619	0.239286	1.79554	1.78055
134	2	0.328571	0.25	0.22619	0.239286	1.79354	1.78457
0.75
135	2	0.285714	0.25	0.25	0.25	1.79244	1.77819
0.75
136	2	0.328571	0.25	0.25	0.257143	1.78944	1.78253
137	2	0.2	0.214286	0.25	0.257143	1.79018	1.78853
138	2	0.185714	0.25	0.25	0.257143	1.79063	1.78595
139	2	0.328571	0.178571	0.25	0.257143	1.79304	1.77842
140	2	0.257143	0.178571	0.25	0.257143	1.7934	1.7861
141	2	0.214286	0.107143	0.25	0.257143	1.79532	1.77954
142	2	0.314286	0.142857	0.25	0.257143	1.79551	1.77286
143	2	0.371429	0.178571	0.25	0.257143	1.79206	1.78167
144	2	0.328571	0.178571	0.25	0.257143	1.79293	1.78294
145	2	0.342857	0.178571	0.25	0.257143	1.79349	1.77681
146	2	0.2	0.178571	0.25	0.2571

0.7857142835855484
262	2	0.214286	0.214286	0.214286	0.242857	1.76823	1.77239
0.7976190447807312
263	2	0.314286	0.178571	0.202381	0.242857	1.76816	1.76655
0.809523805975914
264	2	0.3	0.178571	0.190476	0.242857	1.76779	1.7553
0.8214285671710968
265	2	0.242857	0.178571	0.178571	0.242857	1.76697	1.75919
266	2	0.257143	0.178571	0.178571	0.242857	1.76758	1.7522
267	2	0.3	0.178571	0.178571	0.242857	1.76708	1.74725
0.8214285671710968
268	2	0.228571	0.178571	0.178571	0.242857	1.76608	1.76522
0.8214285671710968
269	2	0.228571	0.178571	0.178571	0.242857	1.76537	1.76062
0.8214285671710968
270	2	0.285714	0.178571	0.178571	0.242857	1.76347	1.75741
0.809523805975914
271	2	0.271429	0.214286	0.190476	0.242857	1.76328	1.75273
0.809523805975914
272	2	0.371429	0.178571	0.190476	0.242857	1.76299	1.74289
0.7976190447807312
273	2	0.428571	0.214286	0.202381	0.242857	1.76253	1.73456
274	2	0.3	0.214286	0.202381	0.242857	1.76305	1.74248
0.7857142835855484
275	2	0.342857	0.214286	0.214286	0.242857	1.76267	1.75227

...

1263	2	0.385714	0.571429	0.571429	0.439286	1.05162	1.23662
0.45238093535105384
1264	2	0.457143	0.571429	0.547619	0.403571	1.04987	1.10251
0.4285714030265808
1265	2	0.385714	0.571429	0.571429	0.403571	1.04832	1.16533
1266	2	0.557143	0.571429	0.571429	0.403571	1.05026	0.987898
1267	2	0.428571	0.571429	0.571429	0.403571	1.05088	1.18289
1268	2	0.457143	0.571429	0.571429	0.403571	1.05325	1.10308
1269	2	0.528571	0.571429	0.571429	0.403571	1.0538	0.972686
1270	2	0.542857	0.571429	0.571429	0.403571	1.05453	1.01468
1271	2	0.457143	0.571429	0.571429	0.403571	1.05377	1.10629
1272	2	0.4	0.571429	0.571429	0.403571	1.05019	1.13402
1273	2	0.471429	0.5	0.571429	0.403571	1.04922	1.1249
1274	2	0.428571	0.5	0.571429	0.403571	1.05044	1.09062
1275	2	0.542857	0.5	0.571429	0.403571	1.05247	1.04736
1276	2	0.5	0.5	0.571429	0.403571	1.05203	1.04948
1277	2	0.557143	0.5	0.571429	0.403571	1.05307	0.992196
1278	2	0.471429	0.5	0.571429	0.403571	1.05223	1.01485
1279	2	0.4	0.5	0.571429	0.403571	1.05344	1.04871
1280	2	

1415	2	0.428571	0.5	0.571429	0.4	1.03471	1.10168
1416	2	0.328571	0.5	0.571429	0.4	1.03717	1.23938
1417	2	0.514286	0.5	0.571429	0.4	1.03645	1.05018
1418	2	0.514286	0.5	0.571429	0.4	1.03683	1.08055
1419	2	0.5	0.5	0.571429	0.4	1.03664	1.05821
1420	2	0.4	0.5	0.571429	0.4	1.03778	1.14091
1421	2	0.5	0.5	0.571429	0.4	1.03585	1.02309
1422	2	0.471429	0.5	0.571429	0.4	1.03653	1.11409
1423	2	0.371429	0.5	0.571429	0.4	1.03643	1.13647
1424	2	0.471429	0.5	0.571429	0.4	1.03616	1.08143
1425	2	0.528571	0.5	0.571429	0.4	1.03557	0.974184
1426	2	0.471429	0.5	0.571429	0.4	1.03523	1.08012
1427	2	0.471429	0.5	0.571429	0.4	1.03172	1.05092
0.5
1428	2	0.371429	0.5	0.5	0.414286	1.03096	1.16102
1429	2	0.4	0.5	0.5	0.414286	1.03118	1.10882
1430	2	0.457143	0.5	0.5	0.414286	1.03219	1.09258
1431	2	0.542857	0.5	0.5	0.414286	1.03696	1.02269
1432	2	0.514286	0.5	0.5	0.414286	1.03771	1.02679
1433	2	0.457143	0.5	0.5	0.414286	1.03819	1.12854
1434	2	0.5	0.5	0.5	0.414286	1.0421	1.05688
1435	2	0.442857	0.5	0.5	0.414286	1.03891	

1581	2	0.285714	0.5	0.5	0.414286	1.02673	1.14942
1582	2	0.485714	0.5	0.5	0.414286	1.02404	1.08119
1583	2	0.442857	0.5	0.5	0.414286	1.02533	1.04859
1584	2	0.442857	0.5	0.5	0.414286	1.02815	1.06155
1585	2	0.628571	0.5	0.5	0.414286	1.02886	0.947539
1586	2	0.514286	0.5	0.5	0.414286	1.03044	1.00915
1587	2	0.4	0.5	0.5	0.414286	1.03061	1.09506
1588	2	0.442857	0.5	0.5	0.414286	1.0303	0.98871
1589	2	0.5	0.5	0.5	0.414286	1.03111	1.06639
1590	2	0.328571	0.5	0.5	0.414286	1.02969	1.16022
1591	2	0.528571	0.5	0.5	0.414286	1.02641	1.04125
1592	2	0.542857	0.5	0.5	0.414286	1.0223	1.00647
1593	2	0.442857	0.5	0.5	0.414286	1.02102	1.06425
1594	2	0.557143	0.5	0.5	0.414286	1.0239	1.02497
1595	2	0.385714	0.5	0.5	0.414286	1.0258	1.12068
0.476190467675527
1596	2	0.457143	0.571429	0.52381	0.439286	1.02647	1.13213
0.45238093535105384
1597	2	0.485714	0.571429	0.547619	0.439286	1.02337	1.00125
0.45238093535105384
1598	2	0.328571	0.5	0.547619	0.417857	1.02282	1.16341
1599	2	0.442857	0.5	0.547619	0.417857	1.02688	1.1

1745	2	0.485714	0.5	0.5	0.414286	1.01488	0.996283
1746	2	0.428571	0.5	0.5	0.414286	1.01755	1.02892
1747	2	0.428571	0.5	0.5	0.414286	1.01828	0.977492
1748	2	0.442857	0.5	0.5	0.414286	1.01785	0.938337
1749	2	0.414286	0.5	0.5	0.414286	1.01915	1.05529
1750	2	0.5	0.5	0.5	0.414286	1.0168	1.09714
1751	2	0.457143	0.5	0.5	0.414286	1.01519	1.08581
1752	2	0.385714	0.5	0.5	0.414286	1.01734	1.11186
1753	2	0.485714	0.5	0.5	0.414286	1.02058	1.12845
1754	2	0.5	0.5	0.5	0.414286	1.01716	1.05521
0.476190467675527
1755	2	0.5	0.571429	0.52381	0.446429	1.01401	1.02218
1756	2	0.485714	0.5	0.52381	0.446429	1.01464	1.10051
1757	2	0.542857	0.5	0.52381	0.446429	1.0164	1.05778
1758	2	0.5	0.5	0.52381	0.446429	1.01779	1.01279
1759	2	0.514286	0.5	0.52381	0.446429	1.02207	1.1042
1760	2	0.428571	0.5	0.52381	0.446429	1.01947	1.09991
1761	2	0.485714	0.5	0.52381	0.446429	1.01708	1.06903
1762	2	0.557143	0.5	0.52381	0.446429	1.0162	1.01471
1763	2	0.485714	0.5	0.52381	0.446429	1.01876	1.06946
1764	2	0.514286	0.5	0.52381	0.4

1898	2	0.414286	0.5	0.52381	0.414286	1.02455	1.08964
1899	2	0.471429	0.5	0.52381	0.414286	1.02463	1.03474
1900	2	0.5	0.5	0.52381	0.414286	1.02305	1.02299
1901	2	0.485714	0.5	0.52381	0.414286	1.01838	1.05058
1902	2	0.485714	0.5	0.52381	0.414286	1.01679	0.96923
1903	2	0.514286	0.5	0.52381	0.414286	1.01323	1.0677
1904	2	0.471429	0.5	0.52381	0.414286	1.01087	1.04821
0.5
1905	2	0.4	0.5	0.5	0.414286	1.00816	1.12757
1906	2	0.457143	0.5	0.5	0.414286	1.009	1.11549
1907	2	0.5	0.5	0.5	0.414286	1.00919	0.992319
1908	2	0.457143	0.5	0.5	0.414286	1.01024	1.09678
1909	2	0.5	0.5	0.5	0.414286	1.01195	1.0221
1910	2	0.5	0.5	0.5	0.414286	1.01366	0.984113
1911	2	0.442857	0.5	0.5	0.414286	1.01667	1.13728
1912	2	0.457143	0.5	0.5	0.414286	1.01487	0.999879
1913	2	0.571429	0.5	0.5	0.414286	1.01452	0.997472
1914	2	0.5	0.5	0.5	0.414286	1.01578	0.989162
1915	2	0.5	0.5	0.5	0.414286	1.0163	0.995278
1916	2	0.542857	0.5	0.5	0.414286	1.01597	1.00882
1917	2	0.5	0.5	0.5	0.414286	1.01571	1.02966
1918	2	0.471429	0.5	0.5	0.4